In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [0]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score
import time
%matplotlib inline

pal = sns.color_palette()

In [0]:
#Load train and test CSVs
df_train = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
df_test = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')

In [0]:
df_train.head()

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import plot_confusion_matrix
# Since this is a multi lable task and the labels are given as tags in a single dataframe series
biner = MultiLabelBinarizer()
tags = df_train['tags'].str.split()
y = biner.fit_transform(tags)

labels = biner.classes_
print('Number of labels: ', len(labels))
print(labels)

In [0]:
from itertools import chain
tag_list = list(chain.from_iterable([tags.split(" ") for tags in df_train['tags'].values]))
tag_set = set(tag_list)

In [0]:
# Histogram of label instances
tag_s = pd.Series(tag_list).value_counts() # To sort them by count
fig, ax = plt.subplots(figsize=(16, 8))
sns.barplot(x=tag_s, y=tag_s.index, orient='h')

In [0]:
#View some of the train images
new_style = {'grid': False}
plt.rc('axes', **new_style)
_, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(20, 20))
i = 0
for f, l in df_train[:9].values:
    img = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))

    ax[i // 3, i % 3].imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    ax[i // 3, i % 3].set_title('{} - {}'.format(f, l))
    #ax[i // 4, i % 4].show()
    i += 1
    
plt.show()

In [0]:
img_resize = (64, 64) # The resize size of each image ex: (64, 64) or None to use the default image size
validation_split_size = 0.1

In [0]:
labels = df_train['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int) #dictionary containing each individual label
for l in labels:
    for l2 in l:
        counts[l2] += 1
t_list=list(counts.keys())

In [0]:
#Create a dictionary assigning a numerical value to each label
label_map = {i:j for j, i in enumerate(t_list)}
label_map

In [0]:
labels_test = df_test['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts_test = defaultdict(int)
for l in labels_test:
    for l2 in l:
        counts_test[l2] += 1

t_list_test=list(counts_test.keys())
label_map1 = {i:j for j, i in enumerate(t_list_test)}
label_map1

In [0]:
# One hot encode the training labels. Convert the images into pixels and resize them
X_train, Y_train = [], []
for img, label in tqdm(df_train.values, miniters = 1000):
  target = np.zeros(17)
  for tag in label.split(' '):
    target[label_map[tag]]=1
  X_train.append(cv2.resize(cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(img)),(64,64)))
  Y_train.append(target)

In [0]:
#Change lists to numpy arrays and normalize
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)
#x_test = np.array(X_test, np.float16)/255

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [0]:
#convert the test images to pixels and resize them as well
X_test=[]
for img, label in tqdm(df_test[:40669].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(img)), (64,64)))
for img, label in tqdm(df_test[40669:].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img)), (64,64)))

In [0]:
import sys
sys.getsizeof(X_train)

In [0]:
def Amazon_Model(input_shape,weight_path):
    model = Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
        
    model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))
    if(weight_path!=None):
        if os.path.isfile(weight_path):
            model.load_weights(weight_path)
    return model

In [0]:
model = Amazon_Model((64, 64,3),None)

In [0]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=128,callbacks=callbacks,shuffle=True)

In [0]:
ynew_test = []
ynew_train = []

In [0]:
x_test = np.array(X_test, np.float16)/255

In [0]:
p_test = model.predict(x_test, batch_size = 128) #save the test predictions
ynew_test.append(p_test)

result = np.array(ynew_test[0])
result = pd.DataFrame(result, columns = labels)
result

In [0]:
result['id'] = df_test['id']
result.to_csv("submission.csv", index=False)